In [2]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from swiss_rounds_simulator.utils import simulate_tournament, simulate_n_tournaments, compare_settings
from swiss_rounds_simulator.utils import  initiate_league, assign_opponents, play_round
from swiss_rounds_simulator.main import run_simulations


New LDC format places of interest :
* The 8 first teams qualify directly to round of 16
* From 9 to 24 : pseudo round of 32
* Interesting teams : i (9th) and y(25th)  

Format :
* 36 teams,
* 8 games,
* Hat determination of the first 2 games

LoL :
* 16 teams,
* Top 8 to Quarter finales,
* 6 games (not exactly, teams with 3 win qualifies, team with 3 loss gets eliminated)


In [3]:
df = pd.read_csv('data/PROTECTED_prob_lin_T=36_R=8.csv').set_index('team')
df

,Id,Level,control_Avg_WR,control_Avg_Rank,control_Thres_8,control_Thres_24,S_L1_Avg_WR,S_L1_Avg_Rank,S_L1_Thres_8,S_L1_Thres_24,S_L2_Avg_WR,S_L2_Avg_Rank,S_L2_Thres_8,S_L2_Thres_24,S_L3_Avg_WR,S_L3_Avg_Rank,S_L3_Thres_8,S_L3_Thres_24
team,,,,,,,,,,,,,,,,,,
a,1,1.00,0.6620,9.60,0.60,0.95,0.5750,14.65,0.35,0.90,0.5995,13.30,0.35,0.90,0.4570,23.00,0.00,0.60
b,2,0.97,0.6930,8.65,0.55,1.00,0.5250,17.25,0.10,0.85,0.5670,15.00,0.20,0.80,0.4995,19.65,0.05,0.70
c,3,0.94,0.6175,11.55,0.45,0.95,0.6050,11.80,0.45,0.90,0.5370,17.50,0.20,0.85,0.4880,21.95,0.00,0.75
d,4,0.92,0.6425,9.95,0.50,1.00,0.5545,15.40,0.20,0.85,0.5430,17.20,0.15,0.80,0.4460,25.05,0.00,0.45
e,5,0.89,0.6370,12.00,0.30,0.90,0.5865,13.55,0.20,0.95,0.5120,18.25,0.10,0.85,0.5000,19.85,0.00,0.85
f,6,0.86,0.6360,10.80,0.35,1.00,0.5745,14.20,0.25,0.85,0.4880,20.90,0.05,0.65,0.4695,21.85,0.00,0.65
g,7,0.83,0.6365,10.35,0.45,0.95,0.5980,13.10,0.25,0.90,0.4940,20.85,0.05,0.75,0.4570,22.80,0.05,0.55
h,8,0.81,0.5925,13.45,0.30,0.90,0.5600,14.65,0.05,0.85,0.5175,17.80,0.10,0.80,0.4620,22.60,0.00,0.60
i,9,0.78,0.5805,13.65,0.45,0.85,0.5555,14.85,0.30,0.85,0.5305,17.05,0.20,0.80,0.4455,24.00,0.00,0.60


In [4]:
cols = ['Id','control_Thres_8'] + ['S_L'+str(i)+'_Thres_8' for i in range(1,4)]
df[cols]

,Id,control_Thres_8,S_L1_Thres_8,S_L2_Thres_8,S_L3_Thres_8
team,,,,,
a,1,0.60,0.35,0.35,0.00
b,2,0.55,0.10,0.20,0.05
c,3,0.45,0.45,0.20,0.00
d,4,0.50,0.20,0.15,0.00
e,5,0.30,0.20,0.10,0.00
f,6,0.35,0.25,0.05,0.00
g,7,0.45,0.25,0.05,0.05
h,8,0.30,0.05,0.10,0.00
i,9,0.45,0.30,0.20,0.00


In [18]:
n_teams = 36
n_rounds = 8

In [24]:
lt = simulate_tournament(nb_teams=n_teams, nb_games=n_rounds, method='probabilistic', delta_level='linear', verbose = False, strategies={'i':[1]})
for i in range(n_rounds+1):
    sub_lt = lt.query(f"Nb_win == {n_rounds-i}")
    if sub_lt.shape[0] != 0:
        display(sub_lt)

,Id,Level,Strategy,Nb_win,Nb_loss,Win_rate,OWR,R1_opp,R1_res,R2_opp,...,R4_res,R5_opp,R5_res,R6_opp,R6_res,R7_opp,R7_res,R8_opp,R8_res,Rank
f,6,0.861111,[],7,1,0.88,0.62,aa,Win,w,...,Win,l,Win,a,Win,b,Win,j,Loss,1
l,12,0.694444,[],7,1,0.88,0.64,ai,Win,j,...,Win,f,Loss,w,Win,o,Win,c,Win,2


,Id,Level,Strategy,Nb_win,Nb_loss,Win_rate,OWR,R1_opp,R1_res,R2_opp,...,R4_res,R5_opp,R5_res,R6_opp,R6_res,R7_opp,R7_res,R8_opp,R8_res,Rank
o,15,0.611111,[],6,2,0.75,0.58,q,Loss,ag,...,Win,j,Win,v,Win,l,Loss,a,Win,3
b,2,0.972222,[],6,2,0.75,0.62,c,Win,v,...,Win,n,Win,p,Win,f,Loss,w,Win,4
j,10,0.75,[],6,2,0.75,0.59,a,Win,l,...,Win,o,Loss,s,Win,m,Win,f,Win,5


,Id,Level,Strategy,Nb_win,Nb_loss,Win_rate,OWR,R1_opp,R1_res,R2_opp,...,R4_res,R5_opp,R5_res,R6_opp,R6_res,R7_opp,R7_res,R8_opp,R8_res,Rank
w,23,0.388889,[],5,3,0.62,0.61,i,Win,f,...,Win,v,Win,l,Loss,p,Win,b,Loss,6
a,1,1.0,[],5,3,0.62,0.59,j,Loss,h,...,Win,c,Win,f,Loss,r,Win,o,Loss,7
c,3,0.944444,[],5,3,0.62,0.58,b,Loss,ah,...,Win,a,Loss,n,Win,i,Win,l,Loss,8
n,14,0.638889,[],5,3,0.62,0.55,k,Win,m,...,Win,b,Loss,c,Loss,aa,Win,y,Win,9
m,13,0.666667,[],5,3,0.62,0.53,ah,Win,n,...,Loss,e,Win,g,Win,j,Loss,r,Win,10
p,16,0.583333,[],5,3,0.62,0.58,ae,Win,y,...,Loss,t,Win,b,Loss,w,Loss,ad,Win,11
g,7,0.833333,[],5,3,0.62,0.53,ag,Win,d,...,Loss,ae,Win,m,Loss,k,Win,h,Win,12
i,9,0.777778,[1],5,3,0.62,0.48,w,Loss,x,...,Win,d,Win,t,Win,c,Loss,s,Win,13


,Id,Level,Strategy,Nb_win,Nb_loss,Win_rate,OWR,R1_opp,R1_res,R2_opp,...,R4_res,R5_opp,R5_res,R6_opp,R6_res,R7_opp,R7_res,R8_opp,R8_res,Rank
r,18,0.527778,[],4,4,0.5,0.53,ab,Win,u,...,Loss,aa,Win,q,Win,a,Loss,m,Loss,14
h,8,0.805556,[],4,4,0.5,0.44,aj,Loss,a,...,Win,u,Win,ae,Win,d,Win,g,Loss,15
s,19,0.5,[],4,4,0.5,0.50,e,Win,aj,...,Loss,y,Win,j,Loss,q,Win,i,Loss,16
y,25,0.333333,[],4,4,0.5,0.45,ad,Win,p,...,Win,s,Loss,z,Win,t,Win,n,Loss,17
ad,30,0.194444,[],4,4,0.5,0.48,y,Loss,z,...,Loss,aj,Win,ab,Win,v,Win,p,Loss,18
t,20,0.472222,[],4,4,0.5,0.56,ac,Win,q,...,Loss,p,Loss,i,Loss,y,Loss,z,Win,19
v,22,0.416667,[],4,4,0.5,0.59,x,Win,b,...,Win,w,Loss,o,Loss,ad,Loss,aa,Win,20
q,17,0.555556,[],4,4,0.5,0.52,o,Win,t,...,Loss,ab,Win,r,Loss,s,Loss,x,Win,21
k,11,0.722222,[],4,4,0.5,0.45,n,Loss,ai,...,Loss,af,Win,e,Win,g,Loss,d,Win,22
u,21,0.444444,[],4,4,0.5,0.38,af,Win,r,...,Loss,h,Loss,ah,Win,ac,Win,e,Win,23


,Id,Level,Strategy,Nb_win,Nb_loss,Win_rate,OWR,R1_opp,R1_res,R2_opp,...,R4_res,R5_opp,R5_res,R6_opp,R6_res,R7_opp,R7_res,R8_opp,R8_res,Rank
e,5,0.888889,[],3,5,0.38,0.48,s,Loss,aa,...,Loss,m,Loss,k,Loss,af,Win,u,Loss,24
d,4,0.916667,[],3,5,0.38,0.45,z,Win,g,...,Win,i,Loss,aj,Win,h,Loss,k,Loss,25
z,26,0.305556,[],3,5,0.38,0.48,d,Loss,ad,...,Loss,ac,Win,y,Loss,ab,Win,t,Loss,26
aa,27,0.277778,[],3,5,0.38,0.45,f,Loss,e,...,Win,r,Loss,x,Win,n,Loss,v,Loss,27
ab,28,0.25,[],3,5,0.38,0.45,r,Loss,ac,...,Loss,q,Loss,ad,Loss,z,Loss,ai,Win,28
x,24,0.361111,[],3,5,0.38,0.44,v,Loss,i,...,Win,ag,Win,aa,Loss,ae,Win,q,Loss,29
ae,31,0.166667,[],3,5,0.38,0.41,p,Loss,af,...,Win,g,Loss,h,Loss,x,Loss,ag,Win,30
ac,29,0.222222,[],3,5,0.38,0.31,t,Loss,ab,...,Win,z,Loss,af,Win,u,Loss,aj,Win,31


,Id,Level,Strategy,Nb_win,Nb_loss,Win_rate,OWR,R1_opp,R1_res,R2_opp,...,R4_res,R5_opp,R5_res,R6_opp,R6_res,R7_opp,R7_res,R8_opp,R8_res,Rank
ag,33,0.111111,[],2,6,0.25,0.42,g,Loss,o,...,Loss,x,Loss,ai,Loss,ah,Win,ae,Loss,32
ah,34,0.083333,[],2,6,0.25,0.39,m,Loss,c,...,Loss,ai,Win,u,Loss,ag,Loss,af,Win,33
ai,35,0.055556,[],2,6,0.25,0.39,l,Loss,k,...,Loss,ah,Loss,ag,Win,aj,Win,ab,Loss,34


,Id,Level,Strategy,Nb_win,Nb_loss,Win_rate,OWR,R1_opp,R1_res,R2_opp,...,R4_res,R5_opp,R5_res,R6_opp,R6_res,R7_opp,R7_res,R8_opp,R8_res,Rank
aj,36,0.027778,[],1,7,0.12,0.45,h,Win,s,...,Loss,ad,Loss,d,Loss,ai,Loss,ac,Loss,35
af,32,0.138889,[],1,7,0.12,0.44,u,Loss,ae,...,Loss,k,Loss,ac,Loss,e,Loss,ah,Loss,36


In [27]:
simu = simulate_n_tournaments(n_tournaments=10,nb_teams=n_teams, nb_games= n_rounds, thresholds= [8, 25],delta_level='linear')
simu

,Id,Level,Strategy,Avg_WR,Avg_Rank,Thres_8,Thres_25
a,1,1.00,[],0.644545,10.1,0.3,0.8
b,2,0.97,[],0.714545,13.2,0.1,0.7
c,3,0.94,[],0.850909,9.6,0.4,0.8
d,4,0.92,[],0.886364,14.6,0.3,0.7
e,5,0.89,[],1.068182,8.6,0.3,0.8
f,6,0.86,[],1.079091,13.9,0.3,0.6
g,7,0.83,[],1.215455,11.2,0.6,0.7
h,8,0.81,[],1.295455,11.8,0.3,0.7
i,9,0.78,[],1.441818,7.4,0.6,0.8
j,10,0.75,[],1.498182,9.1,0.6,0.7
